In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.stats import pearsonr
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.spatial.distance import pdist

from gsn.perform_gsn import perform_gsn

from manifold_dynamics.raw_raster import process_session
import manifold_dynamics.process_data as prcs

In [ ]:
x = np.eye(4)
triu = np.triu_indices_from(x, k=0)
x[triu]

In [ ]:
DATADIR = './../../datasets/NNN/'
uid_sheet = pd.read_csv(os.path.join(DATADIR, 'roi-uid.csv'))
unique_rois = uid_sheet['uid'].unique()

In [ ]:
roi_uid = unique_rois[47]
out = process_session(roi_uid) # shape is (units, 450, 1072, reps)

In [ ]:
# PERFORM GSN ON THE SINGLE ROI/SESSION
rows = []

step = 5
for t in tqdm(range(0, out.shape[1] - step, step)):
    Xw = out[:, slice(t, t + step), :, :]
    Xavg = np.mean(Xw, axis=1)
    Xavg = Xavg * 1e3

    results = perform_gsn(Xavg, {'wantverbose': False})
    sigcov = results['cSb']
    noisecov = results['cNb']

    triu = np.triu_indices_from(sigcov, k=0)

    rows.append({
        'time': t,
        'type': 'signal',
        'covariance': sigcov[triu]
    })

    rows.append({
        'time': t,
        'type': 'noise',
        'covariance': noisecov[triu])
    })
    
    rows.append({
        'time': t,
        'type': 'ncsnr',
        'covariance': results['ncsnr']
    })

cov_df = pd.DataFrame(rows)

print('mean ncsnr', np.nanmean(results['ncsnr']))
print('cov correlation', pearsonr(noisecov[triu], sigcov[triu]).statistic)

fig,ax = plt.subplots(1,1, figsize=(5,3))
sns.scatterplot(cov_df, x='time', y='covariance', hue='type', ax=ax)

In [ ]:
# CLUSTERED cSb MATRIX
M = np.array(sigcov, copy=True)
np.fill_diagonal(M, 0)

# compute pairwise row distances
# correlation distance is usually best for covariance structure
row_dist = pdist(M, metric='correlation')

Z = linkage(row_dist, method='average')
order = leaves_list(Z)
M_reordered = sigcov[np.ix_(order, order)]

# plot
sns.heatmap(M_reordered, vmin=-0.5, vmax=0.5, square=True)

sns.clustermap(
    sigcov,
    metric='correlation',
    method='average',
    vmin=-0.5,
    vmax=0.5,
    cmap='vlag'
)

In [ ]:
# RAW DATA RASTER FOR LOCALIZER IMAGES
U, T, I, R = out.shape
bin_size = 20

rng = np.random.default_rng(0)
unit_ids = rng.choice(U, size=min(12, U), replace=False)
img_ids = rng.choice(I, size=6, replace=False)

Xb = out
Y = np.nanmean(Xb[unit_ids][:, :, 1000:, :], axis=-1)


for unit in Y:
    fig,ax = plt.subplots(1,1, figsize=(20, 5))
    unit[unit==0] = np.nan
    print(unit.shape)
    sns.heatmap(unit.T, square=True, cmap='binary', vmax=0.1)
    plt.show()